In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

### 코사인 유사도 분석

In [2]:
chairDF = pd.read_csv('./project/chair.csv', index_col=0)
chairDF['image'] 

0      ./static/img/kitchen/chair/chair1.png
1      ./static/img/kitchen/chair/chair2.png
2      ./static/img/kitchen/chair/chair3.png
3      ./static/img/kitchen/chair/chair4.png
4      ./static/img/kitchen/chair/chair5.png
                       ...                  
88    ./static/img/kitchen/chair/chair92.png
89    ./static/img/kitchen/chair/chair93.png
90    ./static/img/kitchen/chair/chair94.png
91    ./static/img/kitchen/chair/chair95.png
92    ./static/img/kitchen/chair/chair96.png
Name: image, Length: 93, dtype: object

In [4]:
# 명사 추출
from konlpy.tag import Okt

okt = Okt()

box = []
for i in tqdm(chairDF['review']):
    a = okt.nouns(i)
    # 하나의 문장으로 합쳐지게 join 시킴
    box.append(' '.join(a))

chairDF['nouns'] = box

100%|██████████| 93/93 [00:16<00:00,  5.49it/s]


In [5]:
chairDF.head(1)

,brand,item,price,image,link,review,nouns
0,영가구,베가 폴라 투명 접이식의자 17colors 대량구매추가할인,"25,600",./static/img/kitchen/chair/chair1.png,https://ohou.se/productions/354736/selling?aff...,['자취방에 의자가 없었어서 옛날부터 사고싶던 투명의자 드디어 샀는데 이뻐요 집 꾸...,자취 방 의자 옛날 투명의자 집 전이 달랑 의자 더 거 화장 의자 사용 원래 스툴 ...


In [6]:
# 중간 저장
chairDF.to_csv('chairDF.csv')

In [7]:
chairDF = pd.read_csv('./chairDF.csv', index_col=0)
chairDF.head(1)

,brand,item,price,image,link,review,nouns
0,영가구,베가 폴라 투명 접이식의자 17colors 대량구매추가할인,"25,600",./static/img/kitchen/chair/chair1.png,https://ohou.se/productions/354736/selling?aff...,['자취방에 의자가 없었어서 옛날부터 사고싶던 투명의자 드디어 샀는데 이뻐요 집 꾸...,자취 방 의자 옛날 투명의자 집 전이 달랑 의자 더 거 화장 의자 사용 원래 스툴 ...


In [8]:
# 단어 카운트 기반의 벡터화
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(chairDF['nouns']) # dtm : 텍스트를 단어의 등장 횟수로 표현된 문서-단어 행렬(document-term matrix)
dtm_chair = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names_out()) # dtm.toarray() : 희소행렬인 dtm을 밀집행렬로 변환
dtm_chair

,가가,가게,가격,가겯,가구,가구라,가구점,가기,가까이,가깜,...,흰색,흰자,히히히,힌지,힌침,힐거,힐것,힐수,힘들엇음,힙니
0,0,0,12,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,17,0,0,0,0,0,1,0,...,2,0,0,0,0,0,0,0,0,0
2,0,0,18,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
3,0,0,10,0,1,0,0,0,0,0,...,3,0,2,0,0,0,0,0,0,0
4,0,0,17,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,0,0,8,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
89,0,0,11,0,2,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
90,0,0,8,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
91,0,0,14,0,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
# dtm 파일 저장
dtm_chair.to_csv('dtm_chair.csv')

---

In [11]:
import pandas as pd 
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma

chairDF = pd.read_csv('./chairDF.csv', index_col=0)
dtm_chair = pd.read_csv('./dtm_chair.csv', index_col=0)

# 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
# a, b : 두 벡터
def make_cosine(a,b):
    return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

kkma = Kkma()
text = '앉아도 꺼지지 않는 폭식한 소파'
strlist = kkma.nouns(text)
text = [' '.join(strlist)]

result = cv.transform(text) # 위에서 fit 시켰기 때문에 transform만 해주면 됨
input_text = result.toarray()

brand_list = np.array(chairDF['brand'])
item_list = np.array(chairDF['item'])
price_list = np.array(chairDF['price'])
image_list = np.array(chairDF['image'])

result_cosine = make_cosine(dtm_chair, input_text.reshape(-1))
result_args = result_cosine.argsort()[::-1] # argsort() : 배열을 정렬했을 때의 인덱스를 반환하는 함수

# 내림차순 정렬 => [result_args][0] : 코사인 유사도가 가장 큼
print(brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0])

나무뜰 코젯 원형 스툴 화장대 의자 6color 41,900 ./static/img/kitchen/chair/chair75.png


In [12]:
# 위의 코드 함수화
def find_chair(text):

    import pandas as pd 
    import numpy as np
    from konlpy.tag import Okt
    from konlpy.tag import Kkma

    chairDF = pd.read_csv('./chairDF.csv', index_col=0)
    dtm_chair = pd.read_csv('./dtm_chair.csv', index_col=0)

    # 코사인 유사도 : 두 벡터 사이 각도의 코사인 값 (-1 ~ 1)
    def make_cosine(a,b):
        return np.dot(a,b) / (np.linalg.norm(a) * np.linalg.norm(b))

    kkma = Kkma()
    search_text = [text]
    strlist = kkma.nouns(search_text[0])
    search_text = [(" ".join(strlist))]

    result = cv.transform(search_text)
    search_words = result.toarray()

    brand_list = np.array(chairDF['brand'])
    item_list = np.array(chairDF['item'])
    price_list = np.array(chairDF['price'])
    image_list = np.array(chairDF['image'])

    result_cosine = make_cosine(dtm_chair, search_words.reshape(-1))

    result_args = result_cosine.argsort()[::-1]
    return brand_list[result_args][0], item_list[result_args][0], price_list[result_args][0], image_list[result_args][0]